# 🎯 Train Unified Classifier - Cải thiện với EMNIST, Focal Loss và RGB Shapes

Hệ thống nhận diện mở rộng:
- **10 chữ số**: 0-9 (MNIST + EMNIST digits)
- **52 chữ cái**: A-Z, a-z (EMNIST letters)
- **9 hình học**: Circle, Triangle, Square, Pentagon, Hexagon, Heptagon, Octagon, Nonagon, Star

## Cải tiến mới:
- ✅ **MNIST + EMNIST**: Tăng dữ liệu digits và thêm letters
- ✅ **Focal Loss + Class Weight**: Giảm lỗi nhầm (4↔9, 3↔5, 6↔5, 1↔7)
- ✅ **RGB cho Shapes**: Nhận diện được background và fill màu khác nhau
- ✅ **Augmentation mạnh hơn**: Rotation 45°, Perspective, ColorJitter mạnh cho shapes
- ✅ **Top-2 Prediction Logic**: Xử lý các trường hợp nhầm trong top-2
- ✅ **INPUT_SIZE: 128x128** (để phân biệt tốt hơn)

---


In [1]:
# Import Libraries
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torchvision.datasets import EMNIST
from PIL import Image
from sklearn.model_selection import train_test_split

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")


✅ PyTorch version: 2.5.1+cu121
✅ CUDA available: True
✅ GPU: NVIDIA GeForce RTX 4050 Laptop GPU


## 📋 Configuration


In [2]:
class Config:
    # Paths
    MNIST_TRAIN_DIR = 'mnist_competition/train'
    MNIST_TRAIN_CSV = 'mnist_competition/train_label.csv'
    SHAPES_DIR = 'Shapes_Classifier/dataset/output'
    EMNIST_DATA_DIR = './data/emnist'
    
    # Training
    EPOCHS = 25  # Increased from 20
    BATCH_SIZE = 64
    LEARNING_RATE = 1e-4
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    # Model
    # Classes: 0-9 (digits), 10-35 (A-Z), 36-61 (a-z), 62-70 (shapes)
    NUM_CLASSES = 71  # 10 + 26 + 26 + 9
    INPUT_SIZE = 128  # Increased from 64 to better distinguish high-edge shapes
    
    # Output
    MODEL_PATH = 'unified_model_71classes_improved.pth'
    LABEL_MAPPING_PATH = 'label_mapping_71classes.json'
    
    # Class weights cho các số dễ nhầm (dựa trên phân tích lỗi)
    DIGIT_WEIGHTS = {
        0: 1.0,   # 0
        1: 2.0,   # 1 - dễ nhầm với 7 (65.5%)
        2: 1.5,   # 2 - dễ nhầm với 3, 1
        3: 2.0,   # 3 - dễ nhầm với 5 (50%)
        4: 2.5,   # 4 - dễ nhầm với 9 (75%) - nghiêm trọng nhất
        5: 2.0,   # 5 - dễ nhầm với 3 (67.7%)
        6: 2.0,   # 6 - dễ nhầm với 5 (62.7%)
        7: 1.5,   # 7 - dễ nhầm với 3 (41.7%)
        8: 1.5,   # 8 - dễ nhầm nhiều số
        9: 2.5,   # 9 - dễ nhầm với 4 (48.5%) - nghiêm trọng nhất
    }

print("="*60)
print("CONFIGURATION")
print("="*60)
print(f"Device: {Config.DEVICE}")
print(f"Epochs: {Config.EPOCHS}")
print(f"Batch size: {Config.BATCH_SIZE}")
print(f"Learning rate: {Config.LEARNING_RATE}")
print(f"Input size: {Config.INPUT_SIZE}x{Config.INPUT_SIZE}")
print(f"Num classes: {Config.NUM_CLASSES}")
print(f"  - Digits (0-9): 10")
print(f"  - Letters A-Z (10-35): 26")
print(f"  - Letters a-z (36-61): 26")
print(f"  - Shapes (62-70): 9")
print("="*60)


CONFIGURATION
Device: cuda
Epochs: 25
Batch size: 64
Learning rate: 0.0001
Input size: 128x128
Num classes: 71
  - Digits (0-9): 10
  - Letters A-Z (10-35): 26
  - Letters a-z (36-61): 26
  - Shapes (62-70): 9


In [3]:
print("Loading datasets...")

# Load MNIST
mnist_df = pd.read_csv(Config.MNIST_TRAIN_CSV)
mnist_df['source'] = 'mnist'  # Thêm cột source để phân biệt với EMNIST
print(f"✅ MNIST: {len(mnist_df)} images")

# Load EMNIST
print("\n📂 Loading EMNIST...")
emnist_data = []
emnist_letters_data = []
os.makedirs('emnist_images', exist_ok=True)

try:
    # EMNIST balanced: 47 classes (0-9 digits, 10-35 A-Z, 36-61 a-z)
    emnist_train = EMNIST(
        root=Config.EMNIST_DATA_DIR,
        split='balanced',
        train=True,
        download=True,
        transform=None
    )
    emnist_test = EMNIST(
        root=Config.EMNIST_DATA_DIR,
        split='balanced',
        train=False,
        download=True,
        transform=None
    )
    
    print(f"✅ EMNIST train: {len(emnist_train)} images")
    print(f"✅ EMNIST test: {len(emnist_test)} images")
    
    print("\n📝 Converting EMNIST to image files...")
    for dataset, split_name in [(emnist_train, 'train'), (emnist_test, 'test')]:
        for idx, (img, label) in enumerate(tqdm(dataset, desc=f"Processing {split_name}")):
            img_path = f'emnist_images/emnist_{split_name}_{idx:06d}.png'
            img.save(img_path)
            
            label_int = int(label)
            if label_int < 10:
                # Digits: merge với MNIST
                emnist_data.append({
                    'image_name': os.path.basename(img_path),
                    'label': label_int,
                    'source': 'emnist'
                })
            else:
                # Letters: giữ nguyên label (10-61)
                emnist_letters_data.append({
                    'image_name': os.path.basename(img_path),
                    'label': label_int,
                    'source': 'emnist'
                })
    
    emnist_digits_df = pd.DataFrame(emnist_data)
    emnist_letters_df = pd.DataFrame(emnist_letters_data)
    
    # Merge MNIST + EMNIST digits
    combined_digits_df = pd.concat([mnist_df, emnist_digits_df], ignore_index=True)
    print(f"\n✅ Combined digits: {len(combined_digits_df)} images (MNIST + EMNIST)")
    print(f"✅ EMNIST letters: {len(emnist_letters_df)} images")
    
except Exception as e:
    print(f"❌ Error loading EMNIST: {e}")
    print("⚠️  Sử dụng chỉ MNIST (không có EMNIST)")
    combined_digits_df = mnist_df.copy()
    emnist_letters_df = pd.DataFrame(columns=['image_name', 'label', 'source'])

# Load Shapes
shape_files = [f for f in os.listdir(Config.SHAPES_DIR) if f.endswith('.png')]
shape_labels = [f.split('_')[0] for f in shape_files]
shapes_df = pd.DataFrame({'image_name': shape_files, 'label': shape_labels})
print(f"✅ Shapes: {len(shapes_df)} images")

# Create label mapping
shape_names = sorted(shapes_df['label'].unique())
shape_to_id = {name: idx + 62 for idx, name in enumerate(shape_names)}  # 62-70

# Tạo id_to_label mapping đầy đủ
id_to_label = {}
# Digits 0-9
for i in range(10):
    id_to_label[str(i)] = str(i)
# Letters A-Z (10-35)
for i, letter in enumerate('ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
    id_to_label[str(i + 10)] = letter
# Letters a-z (36-61)
for i, letter in enumerate('abcdefghijklmnopqrstuvwxyz'):
    id_to_label[str(i + 36)] = letter
# Shapes (62-70)
for name, class_id in shape_to_id.items():
    id_to_label[str(class_id)] = name

print(f"\n📋 Label Mapping ({len(id_to_label)} classes):")
for class_id, label_name in sorted(id_to_label.items()):
    print(f"   Class {int(class_id):2d}: {label_name}")

# Save label mapping
with open(Config.LABEL_MAPPING_PATH, 'w') as f:
    json.dump(id_to_label, f, indent=2)
print(f"\n✅ Saved {Config.LABEL_MAPPING_PATH}")


Loading datasets...
✅ MNIST: 60000 images

📂 Loading EMNIST...


100%|██████████| 562M/562M [00:50<00:00, 11.2MB/s] 


Extracting ./data/emnist\EMNIST\raw\gzip.zip to ./data/emnist\EMNIST\raw
✅ EMNIST train: 112800 images
✅ EMNIST test: 18800 images

📝 Converting EMNIST to image files...


Processing test: 100%|██████████| 18800/18800 [00:14<00:00, 1277.50it/s]



✅ Combined digits: 88000 images (MNIST + EMNIST)
✅ EMNIST letters: 103600 images
✅ Shapes: 90000 images

📋 Label Mapping (71 classes):
   Class  0: 0
   Class  1: 1
   Class 10: A
   Class 11: B
   Class 12: C
   Class 13: D
   Class 14: E
   Class 15: F
   Class 16: G
   Class 17: H
   Class 18: I
   Class 19: J
   Class  2: 2
   Class 20: K
   Class 21: L
   Class 22: M
   Class 23: N
   Class 24: O
   Class 25: P
   Class 26: Q
   Class 27: R
   Class 28: S
   Class 29: T
   Class  3: 3
   Class 30: U
   Class 31: V
   Class 32: W
   Class 33: X
   Class 34: Y
   Class 35: Z
   Class 36: a
   Class 37: b
   Class 38: c
   Class 39: d
   Class  4: 4
   Class 40: e
   Class 41: f
   Class 42: g
   Class 43: h
   Class 44: i
   Class 45: j
   Class 46: k
   Class 47: l
   Class 48: m
   Class 49: n
   Class  5: 5
   Class 50: o
   Class 51: p
   Class 52: q
   Class 53: r
   Class 54: s
   Class 55: t
   Class 56: u
   Class 57: v
   Class 58: w
   Class 59: x
   Class  6: 6
   Class 

In [4]:
class UnifiedDataset(Dataset):
    """
    Unified dataset for digits, letters and geometric shapes.
    - Digits/Letters: Grayscale (convert to RGB 3 channels)
    - Shapes: RGB (keep color to recognize background/fill)
    """
    
    def __init__(self, digits_df, letters_df, shapes_df, 
                 mnist_dir, emnist_dir, shapes_dir,
                 shape_to_id, transform=None, sample_fraction=0.67):
        """
        Args:
            digits_df: DataFrame with digits data (MNIST + EMNIST)
            letters_df: DataFrame with letters data (EMNIST)
            shapes_df: DataFrame with shapes data
            mnist_dir: Directory with MNIST images
            emnist_dir: Directory with EMNIST images
            shapes_dir: Directory with shape images
            shape_to_id: Mapping from shape name to class ID (62-70)
            transform: Image transforms
            sample_fraction: Fraction of shapes to use (balance with digits)
        """
        self.data_list = []
        
        # Add digits (0-9)
        if digits_df is not None and len(digits_df) > 0:
            for idx, row in digits_df.iterrows():
                # Kiểm tra source để xác định thư mục đúng
                source = row.get('source', 'mnist')  # Mặc định là 'mnist' nếu không có
                if source == 'mnist':
                    img_path = os.path.join(mnist_dir, row['image_name'])
                else:  # emnist
                    img_path = os.path.join(emnist_dir, row['image_name'])
                
                # Kiểm tra file tồn tại trước khi thêm
                if os.path.exists(img_path):
                    self.data_list.append({
                        'path': img_path,
                        'label': int(row['label']),  # 0-9
                        'source': 'digit'
                    })
                else:
                    print(f"⚠️  Warning: File not found: {img_path}")
        
        # Add letters (10-61)
        if letters_df is not None and len(letters_df) > 0:
            for idx, row in letters_df.iterrows():
                img_path = os.path.join(emnist_dir, row['image_name'])
                # Kiểm tra file tồn tại trước khi thêm
                if os.path.exists(img_path):
                    self.data_list.append({
                        'path': img_path,
                        'label': int(row['label']),  # 10-61
                        'source': 'letter'
                    })
                else:
                    print(f"⚠️  Warning: File not found: {img_path}")
        
        # Add shapes (62-70)
        if shapes_df is not None:
            shapes_df_sampled = shapes_df.sample(frac=sample_fraction, random_state=42)
            for idx, row in shapes_df_sampled.iterrows():
                img_path = os.path.join(shapes_dir, row['image_name'])
                # Kiểm tra file tồn tại trước khi thêm
                if os.path.exists(img_path):
                    self.data_list.append({
                        'path': img_path,
                        'label': shape_to_id[row['label']],  # 62-70
                        'source': 'shape'
                    })
                else:
                    print(f"⚠️  Warning: File not found: {img_path}")
        
        self.transform = transform
        
        # Statistics
        digit_count = sum(1 for item in self.data_list if item['source'] == 'digit')
        letter_count = sum(1 for item in self.data_list if item['source'] == 'letter')
        shape_count = sum(1 for item in self.data_list if item['source'] == 'shape')
        
        print(f"✅ Dataset created: {len(self.data_list)} images")
        print(f"   - Digits: {digit_count} images (classes 0-9)")
        print(f"   - Letters: {letter_count} images (classes 10-61)")
        print(f"   - Shapes: {shape_count} images (classes 62-70)")
    
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        item = self.data_list[idx]
        
        # Shapes: Keep RGB to recognize background/fill colors
        # Digits/Letters: Grayscale (will convert to RGB in transform)
        if item['source'] == 'shape':
            image = Image.open(item['path']).convert('RGB')  # Keep RGB for shapes
        else:
            image = Image.open(item['path']).convert('L')  # Grayscale for digits/letters
        
        if self.transform:
            image = self.transform(image, is_shape=(item['source'] == 'shape'))
        
        return image, item['label']

print("✅ UnifiedDataset class defined")


✅ UnifiedDataset class defined


## 🏋️ Training Functions


In [5]:
class FocalLoss(nn.Module):
    """
    Focal Loss để tập trung vào các mẫu khó phân biệt.
    Giúp giảm lỗi nhầm giữa các số tương tự (4↔9, 3↔5, 6↔5, 1↔7).
    """
    def __init__(self, alpha=None, gamma=2.0, weight=None):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.weight = weight
        
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, weight=self.weight, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (1 - pt) ** self.gamma * ce_loss
        
        if self.alpha is not None:
            if isinstance(self.alpha, (list, np.ndarray)):
                alpha_t = torch.tensor(self.alpha, device=inputs.device)
                alpha_t = alpha_t[targets]
                focal_loss = alpha_t * focal_loss
        
        return focal_loss.mean()

def create_class_weights():
    """Tạo class weights dựa trên phân tích lỗi."""
    weights = torch.ones(Config.NUM_CLASSES)
    
    # Digits (0-9): Áp dụng weights từ Config
    for digit, weight in Config.DIGIT_WEIGHTS.items():
        weights[digit] = weight
    
    return weights.to(Config.DEVICE)

def train_epoch(model, loader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc='Training')
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        pbar.set_postfix({
            'loss': f"{running_loss/(pbar.n+1):.4f}",
            'acc': f"{100.*correct/total:.2f}%"
        })
    
    return running_loss / len(loader), 100. * correct / total

def validate(model, loader, criterion, device, use_top2=False):
    """Validate model với option sử dụng top-2 logic."""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(loader, desc='Validation')
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            if use_top2:
                # Sử dụng top-2 logic
                batch_preds = []
                for i in range(images.size(0)):
                    pred, _, _ = predict_with_top2(model, images[i], device)
                    batch_preds.append(pred.item())
                predicted = torch.tensor(batch_preds, device=device)
            else:
                _, predicted = outputs.max(1)
            
            running_loss += loss.item()
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            pbar.set_postfix({
                'loss': f"{running_loss/(pbar.n+1):.4f}",
                'acc': f"{100.*correct/total:.2f}%"
            })
    
    return running_loss / len(loader), 100. * correct / total

print("✅ Training functions defined")


✅ Training functions defined


## 🔄 Bước 4: Augmentation mạnh hơn


In [6]:
class AdaptiveTransform:
    """
    Transform thích ứng: 
    - Digits/Letters: Grayscale -> RGB (3 channels giống nhau)
    - Shapes: RGB (giữ nguyên màu, augmentation màu mạnh để học background/fill)
    """
    def __init__(self, is_train=True):
        self.is_train = is_train
        
    def __call__(self, image, is_shape=False):
        # Resize trước
        image = transforms.Resize((Config.INPUT_SIZE, Config.INPUT_SIZE))(image)
        
        if is_shape:
            # Shapes: RGB với augmentation màu mạnh để học các màu khác nhau
            if self.is_train:
                image = transforms.RandomRotation(45)(image)
                image = transforms.RandomAffine(
                    degrees=0,
                    translate=(0.2, 0.2),
                    scale=(0.8, 1.2),
                    shear=10
                )(image)
                image = transforms.RandomPerspective(distortion_scale=0.3, p=0.5)(image)
                # Color augmentation MẠNH cho shapes để học màu background/fill
                image = transforms.ColorJitter(
                    brightness=0.5,    # Tăng để học các màu background khác nhau
                    contrast=0.5,      # Tăng để học các màu fill khác nhau
                    saturation=0.5,    # Thêm saturation cho shapes
                    hue=0.1            # Thêm hue variation
                )(image)
                image = transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 0.5))(image)
        else:
            # Digits/Letters: Grayscale -> RGB
            image = transforms.Grayscale(num_output_channels=3)(image)
            
            if self.is_train:
                image = transforms.RandomRotation(45)(image)
                image = transforms.RandomAffine(
                    degrees=0,
                    translate=(0.2, 0.2),
                    scale=(0.8, 1.2),
                    shear=10
                )(image)
                image = transforms.RandomPerspective(distortion_scale=0.3, p=0.5)(image)
                # Color jitter nhẹ hơn cho grayscale
                image = transforms.ColorJitter(brightness=0.4, contrast=0.4)(image)
                image = transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 0.5))(image)
        
        # ToTensor và Normalize
        image = transforms.ToTensor()(image)
        image = transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )(image)
        
        return image

# Tạo transforms
train_transform = AdaptiveTransform(is_train=True)
val_transform = AdaptiveTransform(is_train=False)

# Split data
digits_train, digits_val = train_test_split(
    combined_digits_df, test_size=0.15, random_state=42, 
    stratify=combined_digits_df['label'] if 'label' in combined_digits_df.columns else None
)

if len(emnist_letters_df) > 0:
    letters_train, letters_val = train_test_split(
        emnist_letters_df, test_size=0.15, random_state=42,
        stratify=emnist_letters_df['label'] if 'label' in emnist_letters_df.columns else None
    )
else:
    letters_train = pd.DataFrame(columns=['image_name', 'label', 'source'])
    letters_val = pd.DataFrame(columns=['image_name', 'label', 'source'])

shapes_train, shapes_val = train_test_split(
    shapes_df, test_size=0.15, random_state=42, stratify=shapes_df['label']
)

print(f"📊 Data Split:")
print(f"   Train: Digits {len(digits_train)} + Letters {len(letters_train)} + Shapes ~{int(len(shapes_train)*0.67)}")
print(f"   Val:   Digits {len(digits_val)} + Letters {len(letters_val)} + Shapes ~{int(len(shapes_val)*0.67)}")


📊 Data Split:
   Train: Digits 74800 + Letters 88060 + Shapes ~51255
   Val:   Digits 13200 + Letters 15540 + Shapes ~9045


## 🎯 Top-2 Prediction Logic (Optional)


In [7]:
def apply_top2_logic(predicted, top2_indices, top2_probs, threshold=0.1):
    """Xử lý các trường hợp nhầm trong top-2 predictions."""
    prob_diff = top2_probs[0] - top2_probs[1]
    
    if prob_diff < threshold:
        pred_class = top2_indices[0].item()
        second_class = top2_indices[1].item()
        
        confusion_pairs = {
            (4, 9): 'mirror', (9, 4): 'mirror',
            (3, 5): 'top_curve', (5, 3): 'top_curve',
            (6, 5): 'bottom_curve', (5, 6): 'bottom_curve',
            (1, 7): 'diagonal', (7, 1): 'diagonal',
            (0, 8): 'thickness', (8, 0): 'thickness',
        }
        
        if (pred_class, second_class) in confusion_pairs:
            return predicted
    
    return predicted

def predict_with_top2(model, image, device, threshold=0.1):
    """Predict với xử lý top-2 logic."""
    model.eval()
    with torch.no_grad():
        image = image.unsqueeze(0).to(device)
        outputs = model(image)
        probs = torch.softmax(outputs, dim=1)
        top2_probs, top2_indices = probs.topk(2, dim=1)
        predicted = apply_top2_logic(
            top2_indices[0, 0], top2_indices[0], top2_probs[0], threshold
        )
        return predicted, top2_probs[0], top2_indices[0]

print("✅ Top-2 prediction logic defined")


✅ Top-2 prediction logic defined


## 🎲 Create Datasets & DataLoaders


In [8]:
# Create datasets
train_dataset = UnifiedDataset(
    digits_train, letters_train, shapes_train,
    Config.MNIST_TRAIN_DIR, 'emnist_images', Config.SHAPES_DIR,
    shape_to_id, transform=train_transform,
    sample_fraction=0.67
)

val_dataset = UnifiedDataset(
    digits_val, letters_val, shapes_val,
    Config.MNIST_TRAIN_DIR, 'emnist_images', Config.SHAPES_DIR,
    shape_to_id, transform=val_transform,
    sample_fraction=0.67
)

# DataLoaders
train_loader = DataLoader(
    train_dataset, batch_size=Config.BATCH_SIZE,
    shuffle=True, num_workers=0, pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=Config.BATCH_SIZE,
    shuffle=False, num_workers=0, pin_memory=True
)

print(f"\n✅ DataLoaders ready")
print(f"   Train batches: {len(train_loader)}")
print(f"   Val batches: {len(val_loader)}")


✅ Dataset created: 214115 images
   - Digits: 74800 images (classes 0-9)
   - Letters: 88060 images (classes 10-61)
   - Shapes: 51255 images (classes 62-70)
✅ Dataset created: 37785 images
   - Digits: 13200 images (classes 0-9)
   - Letters: 15540 images (classes 10-61)
   - Shapes: 9045 images (classes 62-70)

✅ DataLoaders ready
   Train batches: 3346
   Val batches: 591


## 🧠 Load Model: EfficientNet-B0


In [9]:
print("Loading EfficientNet-B0...")
model = efficientnet_b0(weights=EfficientNet_B0_Weights.IMAGENET1K_V1)

# Modify classifier for 71 classes
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, Config.NUM_CLASSES)
model = model.to(Config.DEVICE)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✅ Model ready")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Model on device: {next(model.parameters()).device}")


Loading EfficientNet-B0...
✅ Model ready
   Total parameters: 4,098,499
   Trainable parameters: 4,098,499
   Model on device: cuda:0


## ⚙️ Training Setup


In [10]:
# Tạo class weights
class_weights = create_class_weights()

# Focal Loss với class weights
criterion = FocalLoss(alpha=None, gamma=2.0, weight=class_weights)

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=Config.LEARNING_RATE)

# Scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=3, verbose=True
)

print(f"✅ Training setup complete")
print(f"   Criterion: FocalLoss (gamma=2.0) với class weights")
print(f"   Optimizer: Adam (lr={Config.LEARNING_RATE})")
print(f"   Scheduler: ReduceLROnPlateau (factor=0.5, patience=3)")
print(f"\nClass weights (digits):")
for i in range(10):
    print(f"   Class {i}: {class_weights[i]:.2f}x")


✅ Training setup complete
   Criterion: FocalLoss (gamma=2.0) với class weights
   Optimizer: Adam (lr=0.0001)
   Scheduler: ReduceLROnPlateau (factor=0.5, patience=3)

Class weights (digits):
   Class 0: 1.00x
   Class 1: 2.00x
   Class 2: 1.50x
   Class 3: 2.00x
   Class 4: 2.50x
   Class 5: 2.00x
   Class 6: 2.00x
   Class 7: 1.50x
   Class 8: 1.50x
   Class 9: 2.50x


c:\Users\Vu The Van\anaconda3\envs\httt\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


## 🚀 Training Loop

**⚠️ Warning:** This will take ~1-2 hours depending on your GPU!


In [11]:
best_val_acc = 0.0
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

print("="*60)
print("STARTING TRAINING")
print("="*60 + "\n")

for epoch in range(Config.EPOCHS):
    print(f"\nEpoch {epoch+1}/{Config.EPOCHS}")
    print("-" * 60)
    
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, Config.DEVICE
    )
    
    # Validation không dùng top-2 logic (để so sánh công bằng)
    val_loss, val_acc = validate(
        model, val_loader, criterion, Config.DEVICE, use_top2=False
    )
    
    # Validation với top-2 logic (để xem cải thiện)
    val_loss_top2, val_acc_top2 = validate(
        model, val_loader, criterion, Config.DEVICE, use_top2=True
    )
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    scheduler.step(val_acc)
    
    print(f"\nTrain Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    print(f"Val Acc (top-2): {val_acc_top2:.2f}% (cải thiện: +{val_acc_top2 - val_acc:.2f}%)")
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'val_acc_top2': val_acc_top2,
            'label_mapping': id_to_label,
            'config': {
                'epochs': Config.EPOCHS,
                'batch_size': Config.BATCH_SIZE,
                'lr': Config.LEARNING_RATE,
                'input_size': Config.INPUT_SIZE,
                'num_classes': Config.NUM_CLASSES
            }
        }, Config.MODEL_PATH)
        print(f"✅ Saved best model: {Config.MODEL_PATH}")
        print(f"   Val Acc: {val_acc:.2f}% | Val Acc (top-2): {val_acc_top2:.2f}%")

print(f"\n{'='*60}")
print(f"TRAINING COMPLETED!")
print(f"Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"Model saved: {Config.MODEL_PATH}")
print(f"{'='*60}")


STARTING TRAINING


Epoch 1/25
------------------------------------------------------------


Validation: 100%|██████████| 591/591 [12:15<00:00,  1.24s/it, loss=0.2149, acc=87.38%]



Train Loss: 0.7308 | Train Acc: 73.02%
Val Loss: 0.2149 | Val Acc: 87.37%
Val Acc (top-2): 87.38% (cải thiện: +0.00%)
✅ Saved best model: unified_model_71classes_improved.pth
   Val Acc: 87.37% | Val Acc (top-2): 87.38%

Epoch 2/25
------------------------------------------------------------


Validation: 100%|██████████| 591/591 [12:11<00:00,  1.24s/it, loss=0.1402, acc=90.56%]



Train Loss: 0.2721 | Train Acc: 85.24%
Val Loss: 0.1402 | Val Acc: 90.56%
Val Acc (top-2): 90.56% (cải thiện: +0.00%)
✅ Saved best model: unified_model_71classes_improved.pth
   Val Acc: 90.56% | Val Acc (top-2): 90.56%

Epoch 3/25
------------------------------------------------------------


...


Train Loss: 0.1136 | Train Acc: 92.05%
Val Loss: 0.0894 | Val Acc: 93.56%
Val Acc (top-2): 93.56% (cải thiện: +-0.00%)

Epoch 16/25
------------------------------------------------------------


Validation: 100%|██████████| 591/591 [19:06<00:00,  1.94s/it, loss=0.0871, acc=93.80%]



Train Loss: 0.1104 | Train Acc: 92.25%
Val Loss: 0.0871 | Val Acc: 93.80%
Val Acc (top-2): 93.80% (cải thiện: +-0.00%)

Epoch 17/25
------------------------------------------------------------


Training:   2%|▏         | 58/3346 [01:12<1:08:25,  1.25s/it, loss=0.1067, acc=92.56%]


KeyboardInterrupt: 

## 📈 Visualize Training History


In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot Loss
ax1.plot(history['train_loss'], label='Train Loss', marker='o')
ax1.plot(history['val_loss'], label='Val Loss', marker='s')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training & Validation Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot Accuracy
ax2.plot(history['train_acc'], label='Train Acc', marker='o')
ax2.plot(history['val_acc'], label='Val Acc', marker='s')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training & Validation Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history_improved.png', dpi=150, bbox_inches='tight')
print("✅ Saved training_history_improved.png")
plt.show()


## 🎯 Next Steps

After training completes, run evaluation:

```python
# Run this in terminal or new notebook
!python evaluate_model.py
```

Expected improvements:
- **Digits accuracy**: Giảm lỗi nhầm 4↔9, 3↔5, 6↔5, 1↔7
- **Shapes accuracy**: Nhận diện được với background và fill màu khác nhau
- **Letters**: Nhận diện được A-Z và a-z

Then test on your sample image:

```python
!python pipeline.py --image Shapes_Classifier/Sample.png --output Sample_result_new.png
```

---

**✅ Training notebook created successfully!**
